In [ ]:
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/CS3244 Team37'

x_train = pd.read_csv('./HAPT Data Set/x_train.csv')
y_train = pd.read_csv('./HAPT Data Set/y_train.csv')
train_id = pd.read_csv('./HAPT Data Set/Train/subject_id_train.txt', names = ["Subject_id"])
x_test = pd.read_csv('./HAPT Data Set/x_test.csv')
y_test = pd.read_csv('./HAPT Data Set/y_test.csv')
test_id = pd.read_csv('./HAPT Data Set/Test/subject_id_test.txt', names = ["Subject_id"])

label_dict = {1: "WALKING",
              2: "WALKING_UPSTAIRS",
              3: "WALKING_DOWNSTAIRS",
              4: "SITTING",
              5: "STANDING",
              6: "LAYING",
              7: "STAND_TO_SIT",
              8: "SIT_TO_STAND", 
              9: "SIT_TO_LIE", 
              10: "LIE_TO_SIT",
              11: "STAND_TO_LIE",
              12: "LIE_TO_STAND"}



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1N1tMV1_aehlsbZti5OQ51p3uweXfNaHY/CS3244 Team37


In [ ]:
train_subjects = train_id.Subject_id.unique().tolist()

In [ ]:
temp = y_train.value_counts()
max_count = temp[0]
y_train.head()
concatenated = pd.concat([x_train, train_id, y_train], axis="columns")
concatenated.head()
# concatenated.loc[concatenated['Label'] == 'STANDING'].shape

,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,...,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1,Subject_id,Label
0,0.043580,-0.005970,-0.035054,-0.995381,-0.988366,-0.937382,-0.995007,-0.988816,-0.953325,-0.794796,...,-0.713308,-0.112754,0.030400,-0.464761,-0.018446,-0.841559,0.179913,-0.051718,1,STANDING
1,0.039480,-0.002131,-0.029067,-0.998348,-0.982945,-0.971273,-0.998702,-0.983315,-0.974000,-0.802537,...,-0.860677,0.053477,-0.007435,-0.732626,0.703511,-0.845092,0.180261,-0.047436,1,STANDING
2,0.039978,-0.005153,-0.022651,-0.995482,-0.977314,-0.984760,-0.996415,-0.975835,-0.985973,-0.798477,...,-0.761847,-0.118559,0.177899,0.100699,0.808529,-0.849230,0.180610,-0.042271,1,STANDING
3,0.039785,-0.011809,-0.028916,-0.996194,-0.988569,-0.993256,-0.996994,-0.988526,-0.993135,-0.798477,...,-0.491604,-0.036788,-0.012892,0.640011,-0.485366,-0.848947,0.181907,-0.040826,1,STANDING
4,0.038758,-0.002289,-0.023863,-0.998241,-0.986774,-0.993115,-0.998216,-0.986479,-0.993825,-0.801982,...,-0.702490,0.123320,0.122542,0.693578,-0.615971,-0.848164,0.185124,-0.037080,1,STANDING


#Validation Split by subject_id

In [ ]:
# it says train test split, but we can actually use it for validation split (from the training dataset)
# train/val ratio of 75:25
guest_ids_train, guest_ids_val = train_test_split(train_subjects, test_size=0.25, random_state=123)
print(guest_ids_train)
print(guest_ids_val)

[15, 30, 25, 23, 21, 1, 3, 27, 16, 19, 17, 11, 29, 5, 22]
[7, 14, 8, 28, 6, 26]


#Filter concatenated df by subject id into train and val df


In [ ]:
concatenated_train = concatenated.loc[concatenated['Subject_id'].isin(guest_ids_train)]
print(concatenated_train.shape)
concatenated_val = concatenated.loc[concatenated['Subject_id'].isin(guest_ids_val)]
print(concatenated_val.shape)


(5662, 563)
(2105, 563)


(5662, 561)

In [ ]:
x_train_IDsplit = concatenated_train.iloc[:, :-2]
y_train_IDsplit = concatenated_train.iloc[:, -1:]
x_val_IDsplit = concatenated_val.iloc[:, :-2]
y_val_IDsplit = concatenated_val.iloc[:, -1:]


In [ ]:
y_train_IDsplit.shape

(5662, 1)

#old code for upsampling

In [ ]:
from sklearn.utils import resample

upsampled_result = None

for i in label_dict.values():
  concatenated_class = concatenated.loc[concatenated['Label'] == i]
  concatenated_class_upsampled = resample(concatenated_class, 
                                 replace=True,          # sample with replacement
                                 n_samples=max_count,   # to match majority class
                                 random_state=123)      # reproducible results
  if upsampled_result is None:
    upsampled_result = concatenated_class_upsampled
  else:
    upsampled_result = pd.concat([upsampled_result, concatenated_class_upsampled])

x_train_upsampled = upsampled_result.iloc[:,:-1]
print(x_train_upsampled.shape)
y_train_upsampled = upsampled_result.iloc[:,-1:]
print(y_train_upsampled.shape)

(17076, 561)
(17076, 1)


,tBodyAcc-Mean-1,tBodyAcc-Mean-2,tBodyAcc-Mean-3,tBodyAcc-STD-1,tBodyAcc-STD-2,tBodyAcc-STD-3,tBodyAcc-Mad-1,tBodyAcc-Mad-2,tBodyAcc-Mad-3,tBodyAcc-Max-1,...,fBodyGyroJerkMag-MeanFreq-1,fBodyGyroJerkMag-Skewness-1,fBodyGyroJerkMag-Kurtosis-1,tBodyAcc-AngleWRTGravity-1,tBodyAccJerk-AngleWRTGravity-1,tBodyGyro-AngleWRTGravity-1,tBodyGyroJerk-AngleWRTGravity-1,tXAxisAcc-AngleWRTGravity-1,tYAxisAcc-AngleWRTGravity-1,tZAxisAcc-AngleWRTGravity-1
7119,0.059214,0.015657,-0.046926,-0.159856,-0.363889,-0.411356,-0.198068,-0.396310,-0.531267,0.108468,...,0.184349,-0.080368,-0.466699,-0.232134,-0.564892,0.864226,-0.183468,-0.835475,0.177212,-0.060935
6538,-0.002361,0.005842,-0.021440,-0.400287,-0.466226,-0.511695,-0.467465,-0.494648,-0.588229,0.051339,...,0.225846,-0.247804,-0.622271,0.833712,-0.182967,0.892425,-0.538010,-0.694705,0.232327,0.191555
475,0.026992,0.008731,-0.020186,-0.397459,-0.406205,-0.545171,-0.457165,-0.429838,-0.617032,0.177636,...,0.291580,0.754823,0.690149,0.660902,0.248274,-0.600288,-0.373814,-0.729788,0.270730,0.097559
7296,0.029504,-0.001841,-0.041028,-0.240609,-0.461972,-0.485811,-0.281841,-0.519699,-0.588882,0.043788,...,0.059125,-0.227447,-0.637051,0.318715,-0.891571,-0.642665,0.350881,-0.888126,0.157831,-0.027420
465,0.046806,0.010970,-0.034185,-0.495684,-0.452565,-0.618916,-0.549967,-0.500814,-0.664063,-0.146838,...,0.033451,0.522807,0.336893,-0.160155,-0.473816,0.593418,-0.036016,-0.743567,0.259305,0.100132


In [ ]:
x_train_upsampled.to_csv("./HAPT Data Set/x_train_upsampled.csv", index=False)
y_train_upsampled.to_csv("./HAPT Data Set/y_train_upsampled.csv", index=False)


# Validation Split + upsampling

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample


In [ ]:
y_train_IDsplit.value_counts()

Label             
LAYING                1049
STANDING              1035
SITTING                939
WALKING                892
WALKING_UPSTAIRS       770
WALKING_DOWNSTAIRS     713
STAND_TO_LIE            67
SIT_TO_LIE              56
LIE_TO_SIT              49
LIE_TO_STAND            40
STAND_TO_SIT            34
SIT_TO_STAND            18
dtype: int64

In [ ]:
# upsampling for train_IDsplit
temp = y_train_IDsplit.value_counts()
max_count_train_split = temp[0]
concatenated = pd.concat([x_train_IDsplit, y_train_IDsplit], axis="columns")

upsampled_train_IDsplit_result = None

for i in label_dict.values():
  concatenated_class_train = concatenated.loc[concatenated['Label'] == i]
  concatenated_class_train_upsampled = resample(concatenated_class_train, 
                                 replace=True,          # sample with replacement
                                 n_samples=max_count_train_split,   # to match majority class
                                 random_state=123)      # reproducible results
  if upsampled_train_IDsplit_result is None:
    upsampled_train_IDsplit_result = concatenated_class_train_upsampled
  else:
    upsampled_train_IDsplit_result = pd.concat([upsampled_train_IDsplit_result, concatenated_class_train_upsampled])

x_train_split_upsampled = upsampled_train_IDsplit_result.iloc[:,:-1]
print(x_train_split_upsampled.shape)
y_train_split_upsampled = upsampled_train_IDsplit_result.iloc[:,-1:]
print(y_train_split_upsampled.shape)
# val_IDsplit does not need to be upsampled

In [ ]:
x_train_IDsplit.to_csv("./HAPT Data Set/x_train_ID.csv", index=False)
y_train_IDsplit.to_csv("./HAPT Data Set/y_train_ID.csv", index=False)

In [ ]:
x_train_split_upsampled.to_csv("./HAPT Data Set/x_train_ID_upsampled.csv", index=False)
y_train_split_upsampled.to_csv("./HAPT Data Set/y_train_ID_upsampled.csv", index=False)
x_val_IDsplit.to_csv("./HAPT Data Set/x_val_ID.csv", index=False)
y_val_IDsplit.to_csv("./HAPT Data Set/y_val_ID.csv", index=False)



#Old code for upsampling (incorrect)

In [ ]:
# it says train test split, but we can actually use it for validation split (from the training dataset)
# train/val ratio of 80:20
x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(x_train, y_train, test_size=0.2, random_state=123)


In [ ]:
# upsampling for x_train_split
temp = y_train_split.value_counts()
max_count_train_split = temp[0]
y_train_split.head()
concatenated = pd.concat([x_train_split, y_train_split], axis="columns")
concatenated.head()
concatenated.loc[concatenated['Label'] == 'STANDING'].shape

upsampled_train_split_result = None

for i in label_dict.values():
  concatenated_class_train = concatenated.loc[concatenated['Label'] == i]
  concatenated_class_train_upsampled = resample(concatenated_class_train, 
                                 replace=True,          # sample with replacement
                                 n_samples=max_count_train_split,   # to match majority class
                                 random_state=123)      # reproducible results
  if upsampled_train_split_result is None:
    upsampled_train_split_result = concatenated_class_train_upsampled
  else:
    upsampled_train_split_result = pd.concat([upsampled_train_split_result, concatenated_class_train_upsampled])

x_train_split_upsampled = upsampled_train_split_result.iloc[:,:-1]
print(x_train_split_upsampled.shape)
y_train_split_upsampled = upsampled_train_split_result.iloc[:,-1:]
print(y_train_split_upsampled.shape)

(13716, 561)
(13716, 1)


In [ ]:
# x_val_split should not be upsampled
temp = y_val_split.value_counts()
max_count_val_split = temp[0]
y_val_split.head()
concatenated_val = pd.concat([x_val_split, y_val_split], axis="columns")
concatenated_val.head()
concatenated_val.loc[concatenated_val['Label'] == 'STANDING'].shape

upsampled_val_split_result = None

for i in label_dict.values():
  concatenated_class_val = concatenated_val.loc[concatenated_val['Label'] == i]
  concatenated_class_val_upsampled = resample(concatenated_class_val, 
                                 replace=True,          # sample with replacement
                                 n_samples=max_count_val_split,   # to match majority class
                                 random_state=123)      # reproducible results
  if upsampled_val_split_result is None:
    upsampled_val_split_result = concatenated_class_val_upsampled
  else:
    upsampled_val_split_result = pd.concat([upsampled_val_split_result, concatenated_class_val_upsampled])

x_val_split_upsampled = upsampled_val_split_result.iloc[:,:-1]
print(x_val_split_upsampled.shape)
y_val_split_upsampled = upsampled_val_split_result.iloc[:,-1:]
print(y_val_split_upsampled.shape)

WALKING
WALKING_UPSTAIRS
WALKING_DOWNSTAIRS
SITTING
STANDING
LAYING
STAND_TO_SIT
SIT_TO_STAND
SIT_TO_LIE
LIE_TO_SIT
STAND_TO_LIE
LIE_TO_STAND
(3408, 561)
(3408, 1)


In [ ]:
y_val_split_upsampled.value_counts()


Label             
LAYING                284
LIE_TO_SIT            284
LIE_TO_STAND          284
SITTING               284
SIT_TO_LIE            284
SIT_TO_STAND          284
STANDING              284
STAND_TO_LIE          284
STAND_TO_SIT          284
WALKING               284
WALKING_DOWNSTAIRS    284
WALKING_UPSTAIRS      284
dtype: int64

In [ ]:
x_train_split_upsampled.to_csv("./HAPT Data Set/x_trainsplit_train_upsampled.csv", index=False)
y_train_split_upsampled.to_csv("./HAPT Data Set/y_trainsplit_train_upsampled.csv", index=False)
x_val_split_upsampled.to_csv("./HAPT Data Set/x_trainsplit_val_upsampled.csv", index=False)
y_val_split_upsampled.to_csv("./HAPT Data Set/y_trainsplit_val_upsampled.csv", index=False)



In [ ]:
y_val_split.to_csv("./HAPT Data Set/y_trainsplit_val_imbalanced.csv", index=False)
x_val_split.to_csv("./HAPT Data Set/x_trainsplit_val_imbalanced.csv", index=False)